### Preprosses and Building Inverted Index

In [ ]:
import re
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Initialize stop words and stemmer
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

# Unified Preprocessing Function
def preprocess(input_data):

    input_data = input_data.lower()  # Case folding

    # If input is a single word
    if len(input_data.split()) == 1:
        if input_data in stop_words:
            return None  # Ignore stopwords
        return stemmer.stem(input_data)  # Return stemmed term

    # If input is a text (multiple words)
    tokens = re.findall(r'\b\w+\b', input_data)  # Tokenization
    processed_tokens = [
        stemmer.stem(token) for token in tokens if token not in stop_words
    ]  # Remove stop words and apply stemming
    return processed_tokens

# Parse the Collection
def parse_recipes(filename):
    recipes = {}
    with open(filename, "r", encoding="utf-8") as file:
        lines = file.readlines()
        current_id = None
        title = None
        ingredients = None
        instructions = None
        for line in lines:
            line = line.strip()
            
            # Extract ID
            if line.startswith("id:"):
                if current_id is not None:
                    recipes[current_id] = {
                        "Title": title,
                        "Ingredients": ingredients,
                        "Instructions": instructions
                    }
                current_id = int(line.split(":")[1].strip())
                title = None
                ingredients = []
                instructions = []
            
            # Extract Title
            elif line.startswith("Title:"):
                title = line.split(":")[1].strip()
            
            # Extract Ingredients
            elif line.startswith("Ingredients:"):
                ingredients = line.split(":")[1].strip()
                ingredients = ingredients.replace(" ', '", ", ")  # Ensure correct spacing and commas
                
                # Normalize ingredients into a single string
                if isinstance(ingredients, str):
                    ingredients = ingredients.replace("'", "").replace(",", " ").strip()
            
            # Extract Instructions
            elif line.startswith("Instructions:"):
                instructions = line.split("Instructions:")[1].strip()
            
        # Save the last recipe after exiting the loop
        if current_id is not None:
            recipes[current_id] = {
                "Title": title,
                "Ingredients": ingredients,
                "Instructions": instructions
            }
    
    return recipes

# Build Positional Inverted Index (with aggregated positions)
def build_positional_index(documents):
    index = defaultdict(list)
    term_document_frequency = defaultdict(int)
    
    for doc_id, doc in documents.items():
        # Concatenate the title, ingredients, and instructions into a single string
        all_text = f"{doc['Title']} {doc['Ingredients']} {doc['Instructions']}"
        
        tokens = preprocess(all_text)  # Preprocessing the combined text
        if tokens:  # If tokens are available, proceed with indexing
            # Track unique terms in the document to prevent double-counting
            unique_tokens = set(tokens)
            
            for token in unique_tokens:  # Iterate over unique tokens in the document
                term_document_frequency[token] += 1  # Increment document frequency
                
                positions = []
                # Record the positions for this token in the document
                for position, t in enumerate(tokens, start=1):
                    if t == token:
                        positions.append(str(position))
                
                # Store the positions for this term in the document
                index[token].append((doc_id, positions))
    
    # Construct the final format for the index
    formatted_index = {}
    for token, postings in index.items():
        # Document frequency (df) is the number of documents where the term appears
        df = term_document_frequency[token]
        
        # Format each token's postings
        formatted_postings = []
        for doc_id, positions in postings:
            # Add indentation before each doc_id and its positions
            formatted_postings.append(f"       {doc_id}: {','.join(positions)}")
        
        # Add to the final index in the requested format
        formatted_index[token] = f"{df}\n" + "\n".join(formatted_postings)
    
    return formatted_index




# Save Index to File
def save_index_to_file(index, output_filename):
    with open(output_filename, "w", encoding="utf-8") as file:
        for term, postings in sorted(index.items()):
            file.write(f"{term}: {postings}\n")

# File paths for the collections
collection1_path = "recipes.txt"  # Update with your file path

# Parse each collection separately
collection1 = parse_recipes(collection1_path)

# Build separate positional inverted indexes
index_collection1 = build_positional_index(collection1)

# Save indexes to separate files
save_index_to_file(index_collection1, "recipes_index.txt")

print("Positional inverted indexes created:")
print("- recipes index saved to 'recipes_index.txt'")

### Search Boolean,phrase,proximity

In [ ]:
import re
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Preprocess Function
def preprocess(term):
    ps = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    words = term.lower().split()
    words = [ps.stem(w) for w in words if w not in stop_words]
    return words[0] if len(words) == 1 else words

# Load Index Functions

def load_index(filename):
    # Initialize index and track current term and document frequency
    index = defaultdict(lambda: defaultdict(list))
    term_df = {}  # Dictionary to store document frequency for each term
    current_term = None
    current_df = None
    lines_read = 0  # Track how many document lines we've processed for the current term
    
    # Regular expression to match terms followed by a colon and a number (e.g., "éclair: 2")
    term_pattern = re.compile(r"^[^\d\s][^:]+: \d+$")  # Match any term with df, e.g., "éclair: 2"
    
    # Regular expression to match document ID lines (lines that start with a digit)
    doc_id_pattern = re.compile(r"^\d+: .+")  # Match lines that start with a number (doc ID)
    
    with open(filename, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()  # Remove leading and trailing spaces
            
            # Check if the line contains a valid term and its document frequency (e.g., "éclair: 2")
            if re.match(term_pattern, line):
                term, df = line.split(":")  # Split term and df
                current_term = term.strip()  # Set current term
                current_df = int(df.strip())  # Store df, though it's used to control reading lines
                term_df[current_term] = current_df  # Store df for the current term
                lines_read = 0  # Reset line count for the new term
                
                
            # Check if the line is a valid document ID line (e.g., "8651: 29,52,203")
            # Only process this if a current term is set and the line starts with a digit
            elif re.match(doc_id_pattern, line) and current_term:
                try:
                    doc_id, positions = line.split(":")  # Split by colon
                    doc_id = int(doc_id.strip())  # Convert doc_id to integer
                    positions = list(map(int, positions.split(",")))  # Convert positions to a list of integers
                    
                    # Add to index for the current term
                    index[current_term][doc_id] = positions
                    
                    
                    lines_read += 1  # Increment the line counter for the current term

                    # If we've read the number of lines as specified by df, reset for the next term
                    if lines_read == current_df:
                        current_term = None  # Expect a new term after the specified number of document lines
                        

                except ValueError:
                    print(f"Skipping invalid line: {line}")  # Debugging print for malformed lines

    return index, term_df



# Boolean Search Function
def boolean_search(index, query):
    def precedence(operator):
        if operator == "not":
            return 3
        elif operator == "and":
            return 2
        elif operator == "or":
            return 1
        else:
            return 0

    def get_documents(term):
        term = preprocess(term)
        if isinstance(term, list):
            term = term[0] if term else ""
        return set(index.get(term, {}).keys()) if term else set()

    query = query.lower()
    terms = re.findall(r'\w+|[()!&|]', query)
    stack = []
    operators = []

    for token in terms:
        if token == "(":
            operators.append(token)
        elif token == ")":
            while operators and operators[-1] != "(":
                process_operator(stack, operators.pop(), index)
            operators.pop()
        elif token in {"and", "or", "not"}:
            while operators and precedence(operators[-1]) >= precedence(token):
                process_operator(stack, operators.pop(), index)
            operators.append(token)
        else:
            stack.append(get_documents(token))

    while operators:
        process_operator(stack, operators.pop(), index)

    return stack[0] if stack else set()

def process_operator(stack, operator, index):
    if operator == "and":
        doc2 = stack.pop()
        doc1 = stack.pop()
        stack.append(doc1 & doc2)
    elif operator == "or":
        doc2 = stack.pop()
        doc1 = stack.pop()
        stack.append(doc1 | doc2)
    elif operator == "not":
        doc = stack.pop()
        all_docs = set(doc_id for docs in index.values() for doc_id in docs.keys())
        stack.append(all_docs - doc)

# Phrase Search Function
def phrase_search(index, phrase):
    terms = preprocess(phrase)
    if isinstance(terms, str):
        terms = [terms]
    if not terms:
        return set()
    postings = [index[term] for term in terms if term in index]
    if len(postings) < len(terms):
        return set()
    result = set(postings[0].keys())
    for i in range(1, len(terms)):
        result = result & set(postings[i].keys())
        valid_docs = set()
        for doc_id in result:
            positions1 = postings[i - 1][doc_id]
            positions2 = postings[i][doc_id]
            if any(p2 - p1 == 1 for p1 in positions1 for p2 in positions2):
                valid_docs.add(doc_id)
        result = valid_docs
    return result

# Proximity Search Function
def proximity_search(index, term1, term2, proximity):
    term1 = preprocess(term1)
    term2 = preprocess(term2)
    if isinstance(term1, list):
        term1 = term1[0] if term1 else ""
    if isinstance(term2, list):
        term2 = term2[0] if term2 else ""
    if not term1 or not term2:
        return set()
    if term1 not in index or term2 not in index:
        return set()
    result = set(index[term1].keys()) & set(index[term2].keys())
    valid_docs = set()
    for doc_id in result:
        positions1 = index[term1][doc_id]
        positions2 = index[term2][doc_id]
        if any(abs(p2 - p1) <= proximity for p1 in positions1 for p2 in positions2):
            valid_docs.add(doc_id)
    return valid_docs

# Query Parsing and Execution
def run_queries(index, query_file):
    with open(query_file, "r", encoding="utf-8") as file:
        queries = file.readlines()

    results = {}
    for query in queries:
        query_id, query_text = query.strip().split(" ", 1)
        query_text = query_text.strip()

        if '"' in query_text:  # Phrase search
            results[query_id] = phrase_search(index, query_text.strip('"'))
        elif query_text.startswith("#"):  # Proximity search
            match = re.match(r"#(\d+)\((\w+),\s*(\w+)\)", query_text)
            if match:
                proximity = int(match.group(1))
                term1, term2 = match.group(2), match.group(3)
                results[query_id] = proximity_search(index, term1, term2, proximity)
        elif "AND" in query_text or "OR" in query_text or "NOT" in query_text:  # Boolean search
            results[query_id] = boolean_search(index, query_text)
        else:  # Single term
            term = preprocess(query_text)
            if isinstance(term, list):
                term = term[0] if term else ""
            results[query_id] = set(index[term].keys()) if term in index else set()

    return results

# Save Results Function
def save_results(results, output_file):
    with open(output_file, "w", encoding="utf-8") as file:
        for query_id, docs in results.items():
            for doc in sorted(docs):
                file.write(f"{query_id} {doc}\n")

# Main Script
index_files = "recipes_index.txt"
query_filename = "./Queries/queries.boolean.txt"
output_filename = "./Results/queries.results.boolean.txt"

# Load Index
index, df = load_index(index_files)

# Run Queries
results = run_queries(index, query_filename)

# Save Results
save_results(results, output_filename)

print(f"Query results saved to {output_filename}")

Query results saved to ./Results/queries.results.boolean.txt


### Search TFIDF,VSM,BM25

#### TFIDF

In [18]:
import math
from collections import defaultdict

# TF-IDF Scoring Function
def calculate_tfidf(index, query_terms, total_docs):

    scores = defaultdict(float)

    for term in query_terms:
        if term in index:
            term_df=df.get(term, 0)
 
            idf = math.log(total_docs / term_df) if term_df > 0 else 0

            for doc_id, positions in index[term].items():
                tf = len(positions)  # Term frequency in the document
                tfidf = (1 + math.log(tf)) * idf
                scores[doc_id] += tfidf

    return scores

# Query Processing and Ranked Retrieval
def process_queries(index, query_file, total_docs, output_file):

    with open(query_file, "r", encoding="utf-8") as file:
        queries = file.readlines()

    results = []

    for line in queries:
        query_id, query_text = line.strip().split(" ", 1)
        query_terms = preprocess(query_text)  # Preprocess the query
        if not isinstance(query_terms, list):
            query_terms = [query_terms]

        # Calculate TF-IDF scores
        tfidf_scores = calculate_tfidf(index, query_terms, total_docs)

        # Sort by scores in descending order
        sorted_docs = sorted(tfidf_scores.items(), key=lambda x: x[1], reverse=True)

        # Take only the top 5 results
        top_5_docs = sorted_docs[:5]

        # Append results
        for doc_id, score in top_5_docs:
            results.append(f"{query_id} {doc_id},{score:.4f}")

    # Save results to file
    with open(output_file, "w", encoding="utf-8") as file:
        file.write("\n".join(results))


# Main

index_file = "recipes_index.txt"
query_file = "./Queries/queries.ranked.txt"
output_file = "./Results/queries.resultsTFIDF.ranked.txt"

# Load the index and total documents
index,df = load_index(index_file)
total_docs = len(set(doc_id for docs in index.values() for doc_id in docs.keys()))

# Process queries and save results
process_queries(index, query_file, total_docs, output_file)
print(f"TF-IDF retrieval results saved to '{output_file}'")

TF-IDF retrieval results saved to './Results/queries.resultsTFIDF.ranked.txt'


#### VSM

In [99]:
import math
from collections import defaultdict
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Initialize stop words and stemmer
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

# Unified Preprocessing Function
def preprocess(input_data):
    input_data = input_data.lower()  # Case folding

    # If input is a single word
    if len(input_data.split()) == 1:
        if input_data in stop_words:
            return None  # Ignore stopwords
        return stemmer.stem(input_data)  # Return stemmed term

    # If input is a text (multiple words)
    tokens = re.findall(r'\b\w+\b', input_data)  # Tokenization
    processed_tokens = [
        stemmer.stem(token) for token in tokens if token not in stop_words
    ]  # Remove stop words and apply stemming
    return processed_tokens

def compute_cosine_similarity(query_vector, doc_vector, idf, total_docs, epsilon=1e-10):
    # Cosine similarity between query and document vectors
    numerator = sum(query_vector.get(term, 0) * doc_vector.get(term, 0) for term in query_vector)
    
    # Compute the norms of both the query and document vectors
    query_norm = math.sqrt(sum((query_vector.get(term, 0) * idf) ** 2 for term in query_vector))
    doc_norm = math.sqrt(sum((doc_vector.get(term, 0) * idf) ** 2 for term in doc_vector))
    
    # Avoid division by zero with epsilon
    if query_norm == 0 or doc_norm == 0:
        return 0
    
    # Cosine similarity formula
    denominator = query_norm * doc_norm + epsilon
    return numerator / denominator if denominator != 0 else 0

def process_query(query, index, df, total_docs):
    # Preprocess the query before processing it
    query = preprocess(query)
    if not query:  # If the query is empty after preprocessing (e.g., only stopwords)
        return {}
    
    # Convert the list of tokens to a frequency vector
    query_vector = defaultdict(int)
    for term in query:
        query_vector[term.lower()] += 1
        
    scores = defaultdict(float)
    for term, query_tf in query_vector.items():
        if term in index:
            idf = math.log(total_docs / (df.get(term, 1)))  # Correct IDF formula
            for doc_id in index[term]:
                doc_tf = len(index[term][doc_id])  # TF is the number of positions the term occurs in
                # Document and query vectors are built with weighted TF-IDF
                doc_vector = {term: doc_tf * idf}
                query_vector_normalized = {term: query_tf * idf}
                similarity = compute_cosine_similarity(query_vector_normalized, doc_vector, idf, total_docs)
                scores[doc_id] += similarity  # Add similarity for this term to the document's score
    
    return scores

def process_queries(query_file, index, df, total_docs, output_file):
    with open(query_file, "r", encoding="utf-8") as query_file, open(output_file, "w", encoding="utf-8") as result_file:
        query_id = 1
        for query in query_file:
            query = query.strip()
            scores = process_query(query, index, df, total_docs)
            # Sort the scores in descending order
            sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
            
            for doc_id, score in sorted_scores:
                result_file.write(f"{query_id}. {doc_id}, {score:.4f}\n")
            query_id += 1

    print(f"Results VSM saved to {output_file}")

# File paths
index_file = "recipes_index.txt"
query_file = "./Queries/queries.ranked.txt"
output_file = "./Results/queries.resultsVSM.ranked.txt"

# Load the index and document frequency data
index, df = load_index(index_file)
total_docs = len(set(doc_id for docs in index.values() for doc_id in docs.keys()))

# Process the queries and output the results
process_queries(query_file, index, df, total_docs, output_file)


Results VSM saved to ./Results/queries.resultsVSM.ranked.txt


#### BM25

In [102]:
import math
from collections import defaultdict

# BM25 Parameters
k1 = 1.5
b = 0.75

# Calculate Average Document Length
def calculate_avg_doc_length(index):
    total_length = 0
    total_docs = 0
    for term, docs in index.items():
        for doc_id, positions in docs.items():
            total_length += len(positions)
            total_docs += 1
    return total_length / total_docs if total_docs > 0 else 0

# Compute BM25 Scores
def compute_bm25(index, query, doc_lengths, avg_doc_length, N):
    query_terms = preprocess(query)  # Assuming preprocess is defined elsewhere
    if isinstance(query_terms, str):
        query_terms = [query_terms]
    
    scores = defaultdict(float)

    for term in query_terms:
        if term in index:
            df = len(index[term])  # Document frequency for the term
            idf = math.log10((N - df + 0.5) / (df + 0.5))  # IDF formula

            for doc_id, positions in index[term].items():
                tf = len(positions)  # Term frequency in the document
                doc_len = doc_lengths[doc_id]
                # BM25 formula
                score = (tf / (k1 * (doc_len / avg_doc_length) + tf + 0.5)) * idf
                scores[doc_id] += score

    return scores

# Save Results
def save_results(results, output_file):
    with open(output_file, "w", encoding="utf-8") as file:
        for query_id, docs in results.items():
            for doc_id, score in docs:
                file.write(f"{query_id} {doc_id},{score:.2f}\n")
    print(f"Results BM25 saved to {output_file}")

# Function to process queries and rank documents using BM25
def run_bm25_ranking(index_file, query_file, output_file):
    # Load Index
    index, df = load_index(index_file)

    # Calculate Document Lengths and Average Length
    doc_lengths = defaultdict(int)
    for term, docs in index.items():
        for doc_id, positions in docs.items():
            doc_lengths[doc_id] += len(positions)

    avg_doc_length = calculate_avg_doc_length(index)
    N = len(doc_lengths)  # Total number of documents

    # Run Queries
    with open(query_file, "r", encoding="utf-8") as file:
        queries = file.readlines()

    results = {}
    for query in queries:
        query_id, query_text = query.strip().split(" ", 1)
        query_text = query_text.strip()

        bm25_scores = compute_bm25(index, query_text, doc_lengths, avg_doc_length, N)
        ranked_docs = sorted(bm25_scores.items(), key=lambda x: x[1], reverse=True)
        results[query_id] = [(doc_id, score) for doc_id, score in ranked_docs[:5]]

    # Save Results
    save_results(results, output_file)

# Main
index_file = "recipes_index.txt"
query_file = "./Queries/queries.ranked.txt"
output_file = "./Results/queries.resultsBM25.ranked.txt"
    
run_bm25_ranking(index_file, query_file, output_file)


Results BM25 saved to ./Results/queries.resultsBM25.ranked.txt
